In [22]:
import openai
import json
import os

In [23]:
os.environ["AZURE_OPENAI_KEY"] = input("Enter your API key: ")
os.environ["AZURE_OPENAI_ENDPOINT"] = input("Enter your endpoint: ")
deployment_name = input("Enter your deployment name: ")

openai.api_key = os.getenv("AZURE_OPENAI_KEY")
openai.api_base = os.getenv("AZURE_OPENAI_ENDPOINT") # your endpoint should look like the following https://YOUR_RESOURCE_NAME.openai.azure.com/
openai.api_type = 'azure'
openai.api_version = '2023-07-01-preview' # this may change in the future

In [39]:
# Example dummy function hard coded to return the same weather
# In production, this could be your backend API or an external API
def get_current_weather(location, unit="fahrenheit"):
    """Get the current weather in a given location"""
    weather_info = {
        "location": location,
        "temperature": "72",
        "unit": unit,
        "forecast": ["sunny", "windy"],
    }
    
    return json.dumps(weather_info)

In [40]:
def run_conversation():
    # Step 1: send the conversation and available functions to GPT
    messages = [{"role": "user", "content": "What's the weather like in Boston?"}]
    functions = [
        {
            "name": "get_current_weather",
            "description": "Get the current weather in a given location",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state, e.g. San Francisco, CA",
                    },
                    "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
                },
                "required": ["location"],
            },
        }
    ]
    response = openai.ChatCompletion.create(
        deployment_id = deployment_name,
        messages = messages,
        functions = functions,
        function_call = "auto",  # auto is default, but we'll be explicit
    )
    response_message = response["choices"][0]["message"]
    

    print("--- first gpt response ---")
    print(response)

    # Step 2: check if GPT wanted to call a function
    if response_message.get("function_call"):
        # Step 3: call the function
        # Note: the JSON response may not always be valid; be sure to handle errors
        available_functions = {
            "get_current_weather": get_current_weather,
        }  # only one function in this example, but you can have multiple
        function_name = response_message["function_call"]["name"]
        fuction_to_call = available_functions[function_name]
        function_args = json.loads(response_message["function_call"]["arguments"])
        function_response = fuction_to_call(
            location = function_args.get("location"),
            unit = function_args.get("unit"),
        )

        print("--- function response ---")
        print(function_response)

        # AOAIだとContentが帰ってこないため、そのまま投げるとエラーになる
        response_message.update({"content": None})
        
        # Step 4: send the info on the function call and function response to GPT
        messages.append(response_message)  # extend conversation with assistant's reply
        messages.append(
            {
                "role": "function",
                "name": function_name,
                "content": function_response,
            }
        )  # extend conversation with function response

        print("--- aaaa ----")
        print(messages)
        
        second_response = openai.ChatCompletion.create(
            deployment_id = deployment_name,
            messages=messages,
        )  # get a new response from GPT where it can see the function response
        
        print("--- second gpt response ---")
        print(second_response)
        
        return second_response

In [41]:
response = run_conversation()
print("--- final response ---")
print(response)

--- first gpt response ---
{
  "id": "chatcmpl-7g8cjwAW4G3cfulwiuHW6k2VWgOot",
  "object": "chat.completion",
  "created": 1690277281,
  "model": "gpt-35-turbo",
  "prompt_annotations": [
    {
      "prompt_index": 0,
      "content_filter_results": {
        "hate": {
          "filtered": false,
          "severity": "safe"
        },
        "self_harm": {
          "filtered": false,
          "severity": "safe"
        },
        "sexual": {
          "filtered": false,
          "severity": "safe"
        },
        "violence": {
          "filtered": false,
          "severity": "safe"
        }
      }
    }
  ],
  "choices": [
    {
      "index": 0,
      "finish_reason": "function_call",
      "message": {
        "role": "assistant",
        "function_call": {
          "name": "get_current_weather",
          "arguments": "{\n  \"location\": \"Boston, MA\"\n}"
        }
      },
      "content_filter_results": {}
    }
  ],
  "usage": {
    "completion_tokens": 18,
    "p